# Week 3 Homework

ATTENTION: At the end of the submission form, you will be required to include a link to your GitHub repository or other public code-hosting site. This repository should contain your code for solving the homework. If your solution includes code that is not in file format (such as SQL queries or shell commands), please include these directly in the README file of your repository.

<b><u>Important Note:</b></u> <p> For this homework we will be using the 2022 Green Taxi Trip Record Parquet Files from the New York
City Taxi Data found here: </br> https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page </br>
If you are using orchestration such as Mage, Airflow or Prefect do not load the data into Big Query using the orchestrator.</br> 
Stop with loading the files into a bucket. </br></br>
<u>NOTE:</u> You will need to use the PARQUET option files when creating an External Table</br>

<b>SETUP:</b></br>
Create an external table using the Green Taxi Trip Records Data for 2022. </br>
Create a table in BQ using the Green Taxi Trip Records for 2022 (do not partition or cluster this table). </br>
</p>

### Observe URL patterns

- https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet
- https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet
- https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-11.parquet
- https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-12.parquet

Same as for homework#2, can reuse. Now need to make that block a "common utils block" and pass the 
- base_URL
- colour
- year
- extension

## Question 1: count of records

What is count of records for the 2022 Green Taxi Data??

### Answer 1: `840,402`

- 65,623,481
- 840,402
- 1,936,423
- 253,647

### Load Green Taxi Data in Parquet into DataFrame using Mage

In [ ]:
import io
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import requests
from io import BytesIO

def load_data_from_api(*args, **kwargs):
    """
    Template for loading data from API
    """
    taxi_dtypes = {
                'VendorID': pd.Int64Dtype(),
                'passenger_count': pd.Int64Dtype(),
                'trip_distance': float,
                'RatecodeID':pd.Int64Dtype(),
                'store_and_fwd_flag':str,
                'PULocationID':pd.Int64Dtype(),
                'DOLocationID':pd.Int64Dtype(),
                'payment_type': pd.Int64Dtype(),
                'fare_amount': float,
                'extra':float,
                'mta_tax':float,
                'tip_amount':float,
                'tolls_amount':float,
                'improvement_surcharge':float,
                'total_amount':float,
                'congestion_surcharge':float
            }

    # native date parsing 
    parse_dates = ['lpep_pickup_datetime', 'lpep_dropoff_datetime']

    # setup the vars
    months = range(1,13)
    year = 2022
    colour = 'green'
    base_url="https://d37ci6vzurychx.cloudfront.net/trip-data"

    # Create empty list to store DataFrames
    dataframes = []

    # Iterate through months and download data
    for month in months:
        # print(month)
        
        filename = f"{colour}_tripdata_{year}-{month:02d}.parquet" # .csv.gz / parquet
        print(filename)

        url = f"{base_url}/{filename}"
        print(url)
        
        response = requests.get(url, stream=True)

        if response.status_code == 200:
            # Read Parquet file from the downloaded content
            table = pq.read_table(BytesIO(response.content))
             
            # Convert Parquet table to DataFrame
            df = table.to_pandas()
             
            # Append DataFrame to the list
            dataframes.append(df)
            print(f"Downloaded {filename} successfully!")
        
        else:
            print(f"Failed to download {filename}. Status code: {response.status_code}")

    # Concatenate DataFrames
    return pd.concat(dataframes, ignore_index=True)

In [ ]:
df.info()

In [ ]:
-- QN 1
CREATE OR REPLACE EXTERNAL TABLE `terraform-demo-413613.ny_taxi.external_green_tripdata`
OPTIONS (
  format = 'PARQUET',
  uris = ['gs://magic-zoomcamp-ernest-muli/nyc_green_taxi_data.parquet']
);

In [ ]:
-- QN 1 Create a non partitioned table from external table
CREATE OR REPLACE TABLE `terraform-demo-413613.ny_taxi.green_trips_2022_non_partitioned` AS
SELECT * 
FROM `terraform-demo-413613.ny_taxi.external_green_tripdata`

In [ ]:
-- confirm row count is same from mage
SELECT COUNT(*) FROM `terraform-demo-413613.ny_taxi.external_green_tripdata`;

In [ ]:
-- confirm row count is same external table
SELECT COUNT(*) FROM `terraform-demo-413613.ny_taxi.green_trips_2022_non_partitioned`;

## Question 2: estimated data amount

### Answer 2: `0 MB for the External Table and 6.41MB for the Materialized Table`

Write a query to count the distinct number of PULocationIDs for the entire dataset on both the tables.</br> 
What is the estimated amount of data that will be read when this query is executed on the External Table and the Table?

- 0 MB for the External Table and 6.41MB for the Materialized Table
- 18.82 MB for the External Table and 47.60 MB for the Materialized Table
- 0 MB for the External Table and 0MB for the Materialized Table
- 2.14 MB for the External Table and 0MB for the Materialized Table

In [ ]:
-- qn 2, turn off cache results in query settings, select and view estimate
SELECT DISTINCT(PULocationID) 
FROM `terraform-demo-413613.ny_taxi.external_green_tripdata`;
-- shows 0B estimated, 6.41MB processed, 10MB billed

SELECT DISTINCT(PULocationID) 
FROM `terraform-demo-413613.ny_taxi.green_trips_2022_non_partitioned`;
-- shows 6.41MB estimated, 6.41MB processed, 10MB billed

## Question 3:
### Answer 3: `1,622`
How many records have a fare_amount of 0?
- 12,488
- 128,219
- 112
- 1,622

In [ ]:
-- QN 3 row count for trips with 0 fare_amount
SELECT COUNT(*)
FROM `terraform-demo-413613.ny_taxi.green_trips_2022_non_partitioned`
WHERE fare_amount=0;

## Question 4:
### Answer 4: `Partition by lpep_pickup_datetime  Cluster on PUlocationID`

What is the best strategy to make an optimized table in Big Query if your query will always order the results by PUlocationID and filter based on lpep_pickup_datetime? (Create a new table with this strategy)

- Cluster on lpep_pickup_datetime Partition by PUlocationID
- Partition by lpep_pickup_datetime  Cluster on PUlocationID
- Partition by lpep_pickup_datetime and Partition by PUlocationID
- Cluster on by lpep_pickup_datetime and Cluster on PUlocationID

In [ ]:
-- QN 4 create table with the strategy:
-- Partition by lpep_pickup_datetime  Cluster on PUlocationID
CREATE OR REPLACE TABLE `terraform-demo-413613.ny_taxi.green_trips_2022_optimized`
PARTITION BY DATE(lpep_pickup_dt)
CLUSTER BY PUlocationID AS
SELECT *, 
  TIMESTAMP_MICROS(CAST(lpep_pickup_datetime / 1000 AS INT64)) AS lpep_pickup_dt, 
  TIMESTAMP_MICROS(CAST(lpep_dropoff_datetime / 1000 AS INT64)) AS lpep_dropoff_dt 
FROM `terraform-demo-413613.ny_taxi.green_trips_2022_non_partitioned`;

## Question 5:
### Answer 5: `12.82 MB for non-partitioned table and 1.12 MB for the partitioned table`

Write a query to retrieve the distinct PULocationID between lpep_pickup_datetime
06/01/2022 and 06/30/2022 (inclusive)</br>

Use the materialized table you created earlier in your from clause and note the estimated bytes. Now change the table in the from clause to the partitioned table you created for question 4 and note the estimated bytes processed. What are these values? </br>

Choose the answer which most closely matches.</br> 

- 22.82 MB for non-partitioned table and 647.87 MB for the partitioned table
- 12.82 MB for non-partitioned table and 1.12 MB for the partitioned table
- 5.63 MB for non-partitioned table and 0 MB for the partitioned table
- 10.31 MB for non-partitioned table and 10.31 MB for the partitioned table

In [ ]:
SELECT UNIX_MICROS(TIMESTAMP "2022-06-01 00:00:00-05") AS start_date;
SELECT UNIX_MICROS(TIMESTAMP "2022-07-01 00:00:00-05")  AS end_date;

SELECT 
  DISTINCT(PULocationID)
FROM 
  `terraform-demo-413613.ny_taxi.green_trips_2022_non_partitioned`
WHERE 
  lpep_pickup_datetime >= 1654059600000000
  and 
  lpep_pickup_datetime <= 1656651600000000
  ;

In [ ]:
SELECT 
  DISTINCT(PULocationID)
FROM `terraform-demo-413613.ny_taxi.green_trips_2022_optimized`
WHERE 
  lpep_pickup_dt >= TIMESTAMP("2022-06-01 00:00:00", "America/New_York") and 
  lpep_pickup_dt <= TIMESTAMP("2022-07-01 00:00:00", "America/New_York")
;

## Question 6: 
### Answer 6: `GCP Bucket`

Where is the data stored in the External Table you created?

- Big Query
- GCP Bucket
- Big Table
- Container Registry

## Question 7:
### Answer 7: `False`

It is best practice in Big Query to always cluster your data:
- True
- False

## (Bonus: Not worth points) Question 8:

No Points: Write a `SELECT count(*)` query FROM the materialized table you created. How many bytes does it estimate will be read? Why?

- BigQuery already has the answer so no extra processing is required
- the estimate may be due to caching?